In [1]:
import numpy as np
from numpy import linalg as LA
# ensure 'ncon.py' in working dir
from ncon import ncon

In [2]:
d = 3
A = np.random.rand(d,d,d,d); B = np.random.rand(d,d,d) 
C = np.random.rand(d,d,d); D = np.random.rand(d,d,d) 
E = np.random.rand(d,d,d); F = np.random.rand(d,d,d) 
G = np.random.rand(d,d,d)
# compute density matrices and their principle square roots
rho1 = ncon([B,D,E,B,D,E],[[5,6,-2],[1,2,5],[3,4,6],[7,8,-1],[1,2,7],[3,4,8]])
rho2 = ncon([F,F],[[1,2,-2],[1,2,-1]])
rho3 = ncon([C,G,C,G],[[3,5,-2],[1,2,3],[4,5,-1],[1,2,4]])
d1, u1 = LA.eigh(rho1); sq_d1 = np.sqrt(abs(d1))
d2, u2 = LA.eigh(rho2); sq_d2 = np.sqrt(abs(d2))
d3, u3 = LA.eigh(rho3); sq_d3 = np.sqrt(abs(d3))
X1 = u1 @ np.diag(sq_d1) @ u1.T; X1inv = u1 @ np.diag(1/sq_d1) @ u1.T
X2 = u2 @ np.diag(sq_d2) @ u2.T; X2inv = u2 @ np.diag(1/sq_d2) @ u2.T
X3 = u3 @ np.diag(sq_d3) @ u3.T; X3inv = u3 @ np.diag(1/sq_d3) @ u3.T
# execute gauge changes
Aprime = ncon([A,X1,X2,X3],[[1,-2,2,3],[-1,1],[-3,2],[-4,3]])
Bprime = ncon([B,X1inv],[[-1,-2,1],[1,-3]])
Fprime = ncon([F,X2inv],[[-1,-2,1],[1,-3]])
Cprime = ncon([C,X3inv],[[-1,-2,1],[1,-3]])
# new network is formed from tensors: {Aprime,Bprime,Cprime,D,E,Fprime,G}

# check both networks evaluate to the same tensor
connectlist = [[3,-5,4,5],[1,2,3],[6,-10,5],[-1,-2,1],[-3,-4,2],[-6,-7,4],[-8,-9,6]]
H0 = ncon([A,B,C,D,E,F,G],connectlist)
H1 = ncon([Aprime,Bprime,Cprime,D,E,Fprime,G],connectlist)
dH = LA.norm(H0 - H1) / LA.norm(H0)

In [5]:
X1@X1inv

array([[ 1.00000000e+00,  2.25911908e-15, -2.68558977e-15],
       [-6.06989667e-15,  1.00000000e+00, -3.49796949e-15],
       [-5.02485167e-15,  2.49914785e-15,  1.00000000e+00]])

In [6]:
u1

array([[-0.50126241,  0.73986942, -0.44869727],
       [ 0.72679081,  0.07858842, -0.68234814],
       [-0.46958611, -0.66814452, -0.57712371]])

In [7]:
d1

array([1.97185049e-01, 6.71249971e-01, 2.11702417e+02])

In [8]:
d = 5 # local dimension
chi = 3 # max internal dimension 
H0 = (np.sqrt(1+np.arange(d**7))).reshape(d,d,d,d,d,d,d).transpose(6,5,4,3,2,1,0) 

# first decomposition
utemp,stemp,vhtemp = LA.svd(H0.reshape(d**2,d**5),full_matrices=False)
U0 = (utemp[:,:chi]).reshape(d,d,chi)
H1 = (np.diag(stemp[:chi]) @ vhtemp[:chi,:]).reshape(chi,d,d,d,d,d)
# second decomposition
utemp,stemp,vhtemp = LA.svd(H1.transpose(1,2,0,3,4,5).reshape(d**2,chi*d**3),full_matrices=False)
U1 = (utemp[:,:chi]).reshape(d,d,chi)
H2 = (np.diag(stemp[:chi]) @ vhtemp[:chi,:]).reshape(chi,chi,d,d,d).transpose(1,0,2,3,4)
# third decomposition
utemp,stemp,vhtemp = LA.svd(H2.reshape(chi**2,d**3),full_matrices=False)
U2 = (utemp[:,:chi]).reshape(chi,chi,chi)
H3 = (np.diag(stemp[:chi]) @ vhtemp[:chi,:]).reshape(chi,d,d,d)
# fourth decomposition
utemp,stemp,vhtemp = LA.svd(H3.reshape(chi*d,d**2),full_matrices=False)
V3 = vhtemp[:chi,:].reshape(chi,d,d).transpose(1,2,0)
H4 = (utemp[:,:chi] @ np.diag(stemp[:chi])).reshape(chi,d,chi)
# check result
H0recovered = ncon([U0,U1,U2,V3,H4],[[-1,-2,1],[-3,-4,2],[1,2,3],[-6,-7,4],[3,-5,4]])
totErr = LA.norm(H0 - H0recovered) / LA.norm(H0)

In [9]:
totErr

6.38197335913538e-05

In [10]:
H0

array([[[[[[[  1.        , 125.00399994, 176.7795237 , 216.50866033,
             250.00199999],
            [ 55.91064299, 136.93429081, 185.40765896, 223.60903381,
             256.17572094],
            [ 79.06326581, 147.90537516, 193.65174928, 230.49078073,
             262.20411896],
            [ 96.8297475 , 158.11704525, 201.55892439, 237.17293269,
             268.09699737],
            [111.80787092, 167.70807971, 209.16739708, 243.67191057,
             273.86310449]],

           [[ 25.01999201, 127.4794101 , 178.53851125, 217.94724132,
             251.24888059],
            [ 61.24540799, 139.19770113, 187.08554193, 225.00222221,
             257.39269609],
            [ 82.92164977, 150.0033333 , 195.25880262, 231.84261903,
             263.39324213],
            [100.00499988, 160.08122938, 203.10342193, 238.48689691,
             269.2600973 ],
            [114.56875665, 169.56119839, 210.65611788, 244.95101551,
             275.00181818]],

           [[ 35.36947837,